In [1]:
import ray
GPU_UTILIZATION = 0.0
ray.init(num_gpus=GPU_UTILIZATION)

2023-11-25 16:00:25,403	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-25 16:00:28,723	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.7.1
Dashboard:,http://127.0.0.1:8265


In [2]:
import gymnasium as gym
from ray.tune.registry import register_env
from ray.rllib.env.base_env import BaseEnv

class custom_env(gym.Env):
    def __init__(self,config):
        gym.Env.__init__(self)
        self.env = gym.make('CartPole-v1')
        self.observation_space = self.env.observation_space
        self.action_space = self.env.action_space

    def reset(self,*, seed=None, options=None):
        obs,info = self.env.reset()
        return obs,info

    def step(self, action):
        observation, reward, terminated, truncated,done = self.env.step(action)
        return observation, reward, terminated, truncated,done 

def env_creator(config):
    if len(config) == 0:
        config = {}
    return custom_env(config=config)

register_env("my_env", env_creator)

2023-11-25 16:00:32,814	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-25 16:00:40,675	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
from ray.rllib.algorithms.ppo import PPOConfig

config = PPOConfig().training(gamma=0.9, lr=0.0001, train_batch_size=512, sgd_minibatch_size=256)
config = config.resources(num_gpus=GPU_UTILIZATION, num_learner_workers=1, num_gpus_per_worker=0, num_gpus_per_learner_worker=GPU_UTILIZATION)
config = config.rollouts(num_rollout_workers=4,create_env_on_local_worker=True)

algo = config.build(env="my_env")  

2023-11-25 16:00:46,810	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-11-25 16:00:46,811	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/home/przemek/miniconda3/envs/store21/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarnin

In [4]:
from time import time 
init_eval = algo.evaluate()

t1 = time()
for i in range(32):
    print(i,end='\r')
    algo.train()

print(time() - t1)
post_eval = algo.evaluate()

32.185038328170776


In [5]:
print('Mean reward per episode')
print('Before training: ',init_eval['evaluation']['episode_reward_mean'])
print('After training: ',post_eval['evaluation']['episode_reward_mean'])

Mean reward per episode
Before training:  21.64406779661017
After training:  235.4


In [6]:
import gymnasium as gym 
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [7]:
env = gym.make('CartPole-v1', render_mode="rgb_array")
model = algo.get_policy().model

In [8]:
next_state = env.reset()[0]
num_steps = 120

for step in range(num_steps):
    with torch.no_grad():
        action = model({'obs': torch.Tensor(next_state.reshape([1,4]))})['action_dist_inputs'].argmax().item()

    next_state, reward, done = env.step(action)[:3]

    clear_output(wait=True)
    plt.imshow(env.render())
    plt.show()

    print(step, end='\r')
    
    if done:
        print(f"Episode finished after {step+1} steps")
        break

DependencyNotInstalled: pygame is not installed, run `pip install gymnasium[classic-control]`

In [18]:
env.reset()

(array([ 0.01834047,  0.03332799, -0.0126188 ,  0.03240417], dtype=float32),
 {})